Assignment: Flowers Recognition <br>
Dataset Description:<br>

This dataset contains 4242 images of flowers.<br>
The data collection is based on the data flicr, google images, yandex images.<br>
You can use this datastet to recognize plants from the photo.<br>

Attribute Information:<br>
The pictures are divided into five classes: chamomile, tulip, rose, sunflower, dandelion.<br>
For each class there are about 800 photos. Photos are not high resolution, about 320x240 pixels. <br>
<b>Also explore how to resize images in tensorflow and then resize all the images to a same size. </b> <br>
This is a Multiclass Classification Problem.<br>




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


WORKFLOW : <br>
Load Data <br>
Split into 60 and 40 ratio.<br>
Encode labels.<br>
Create Model<br>
Compilation Step (Note : Its a Multiclass Classification problem , select loss , metrics according to it)<br>
Train the Model.<br>
If the model gets overfit tune your model by changing the units , No. of layers , epochs , add dropout layer or add Regularizer according to the need .<br>
Prediction should be > 85%<br>
Evaluation Step<br>
Prediction<br>




Data : <br>
https://drive.google.com/file/d/1-OX6wn5gA-bJpjPNfSyaYQLz-A-AB_uj/view?usp=sharing

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import pandas as pd
import pathlib
from pathlib import Path
import glob

In [2]:
img_dir0='/content/drive/MyDrive/flowers/daisy'
img_dir1='/content/drive/MyDrive/flowers/dandelion'
img_dir2='/content/drive/MyDrive/flowers/rose'
img_dir3='/content/drive/MyDrive/flowers/sunflower'
img_dir4='/content/drive/MyDrive/flowers/tulip'

In [3]:
img_dir3

'/content/drive/MyDrive/flowers/sunflower'

In [4]:
def read_imgs(img_dir_path):
  for img in glob.glob(img_dir_path+"/*.jpg"):
    img=cv2.imread(img)
    output=cv2.resize(img/255,(150,150))
    yield output

In [5]:
output0=np.array(list(read_imgs(img_dir0)))

In [6]:
output1=np.array(list(read_imgs(img_dir1)))

In [7]:
output2=np.array(list(read_imgs(img_dir2)))

In [8]:
output3=np.array(list(read_imgs(img_dir3)))


In [9]:
output4=np.array(list(read_imgs(img_dir4)))

In [10]:
# giving labels to images
label0=np.zeros((output0.shape[0],1))
label1=np.ones((output1.shape[0],1))
label2=np.ones((output2.shape[0],1))*2
label3=np.ones((output3.shape[0],1))*3
label4=np.ones((output4.shape[0],1))*4

In [11]:
data=np.concatenate((output0,output1,output2,output3,output4))
data= data.reshape(data.shape[0], data.shape[1]*data.shape[2]*data.shape[3])
data.shape

(4323, 67500)

In [12]:
labels=np.concatenate((label0,label1,label2,label3,label4))
labels.shape

(4323, 1)

In [13]:
df=pd.DataFrame(data)
df['labels']=labels.astype('float32')

In [14]:
from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels=train_test_split(df.iloc[0:,:-1], df['labels'], test_size=0.3, random_state=42, stratify = labels)

In [15]:
train_data = np.array(train_data).reshape(len(train_data),150,150,3)
test_data = np.array(test_data).reshape(len(test_data),150,150,3)

In [16]:
train_labels = np.array(train_labels).astype('float32')
test_labels = np.array(test_labels).astype('float32')

In [17]:
from keras.utils import to_categorical
train_labels=to_categorical(train_labels)
test_labels=to_categorical(test_labels)

In [18]:
import keras
img_preprocessing = keras.preprocessing.image.ImageDataGenerator(
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.3,
    zoom_range=0.2,
    channel_shift_range=0.3,
    fill_mode="nearest",
    cval=0.0,
    horizontal_flip=True,
    vertical_flip=False,
    rescale=None,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0,
    dtype=None,
)

In [19]:
img_preprocessing.fit(train_data)
img_preprocessing.fit(test_data)

In [20]:
from keras.models import Sequential
from keras import  layers

In [21]:
train_data.shape[1]

150

In [42]:
model=Sequential()
model.add(layers.Dense(512, activation='relu' , input_shape=(150,150,3,)))
model.add(layers.Dense(5, activation='softmax'))

In [43]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
result = model.fit(
          x= train_data,
          y= train_labels,
          batch_size = 30,
          epochs=100
         
        
        )